In [10]:
def read_BLOSUM_table(path="/Users/marcisin/Desktop/BLOSUM62.txt"):
    """
    Reads BLOSUM table
    the key = tupple (row_char, col_char)
    """
    fr = open(path,"r")
    res_mat = {}
    alpha = fr.readline().strip().split()
    for line in fr.readlines():
        row_alpha = line[0]
        for col_alpha, val in zip(alpha,line.split()[1:]):
            key = (row_alpha, col_alpha)
            res_mat[key] = int(val)
    return res_mat

def read_seq_file(path):
    with open(path) as f:
        seq1 = f.readline().strip()
        seq2 = f.readline().strip()
    return seq1, seq2

In [11]:
def ConstructGlobalAllignment(backtrack,seq1,seq2):
    """
    Constructs globall allignment from the allignment graph
    """
    ALL_1=""
    ALL_2=""
    i = len(seq1)
    j = len(seq2)
    while i > 0 and j > 0:
        if backtrack[i][j] == "DG":
            ALL_1+=seq1[i-1]
            ALL_2+=seq2[j-1]
            i = i-1
            j = j-1
        # Down
        elif backtrack[i][j] == "D":
            ALL_1+=seq1[i-1]
            ALL_2+="-"
            i = i-1
        else:
            ALL_1+="-"
            ALL_2+=seq2[j-1]
            j = j-1
            
   # because we want full allignment, we need to get to point 0,0
    while i != 0 or j != 0:
        if backtrack[i]!=0:
            ALL_1+=seq1[i-1]
            ALL_2+="-"
            i = i-1
        else:
            ALL_1+="-"
            ALL_2+=seq2[j-1]
            j = j-1
    
    print(ALL_1[::-1])
    print(ALL_2[::-1])

In [32]:
def GlobalAllignmentWithAffineGapGraph(seq1, seq2, table, open_penalty, gap_penalty):
    """
    Creates allignment matrices with affine gap
    seq1 -> represents rowumn
    seq2 -> represents col
    indel_panalaty -> shoud be possitive number. We substract in the code.
    Returns (path_graph, max_score)
    """
    # we have 3 matrices
    horizontal_mat = [ [0 for row in range(len(seq2)+1)] for col in range(len(seq1)+1)]
    diagonal_mat = [ [0 for row in range(len(seq2)+1)] for col in range(len(seq1)+1)]
    vertical_mat = [ [0 for row in range(len(seq2)+1)] for col in range(len(seq1)+1)]
    # start with open penalty and than just add
    horizontal_mat[1][0] = open_penalty
    for row in range(2, len(seq1)+1):
        horizontal_mat[row][0] = horizontal_mat[row-1][0] - gap_penalty  
    vertical_mat[0][1] = open_penalty
    for col in range(2, len(seq2)+1):
        vertical_mat[0][col] = vertical_mat[0][col-1] - gap_penalty
    
    
    bt_graph = [ ["-" for row in range(len(seq2)+1)] for col in range(len(seq1)+1)]
    for row in range(1, len(seq1)+1):
        for col in range(1, len(seq2)+1):
            key = (seq1[row-1], seq2[col-1])
            match_score = table[key]
            # when opening a gap we allways use open penalty from the diagonal
            # horizontal, we can either open gap or continue gap, also we can move (look) only in row
            horizontal_mat[row][col] = max( horizontal_mat[row-1][col] - gap_penalty, diagonal_mat[row-1][col] - open_penalty)
            # vertical, we can either open gap or continue gap, also we can move (look) only in column
            vertical_mat[row][col] = max( vertical_mat[row][col-1] - gap_penalty, diagonal_mat[row][col-1] - open_penalty)
            # in diagonal we can either continue or return from gaps
            diagonal_mat[row][col] = max(diagonal_mat[row-1][col-1] + match_score, vertical_mat[row][col], horizontal_mat[row][col])
            
            if horizontal_mat[row][col] >= max(vertical_mat[row][col], diagonal_mat[row][col]):
                # go down
                bt_graph[row][col] = "D"
            elif vertical_mat[row][col] >= max(horizontal_mat[row][col], diagonal_mat[row][col]):
                # go right
                bt_graph[row][col] = "R"
            else:
                # go diagonal
                bt_graph[row][col] = "DG"
    
#     print(bt_graph)
    
    return bt_graph, max(horizontal_mat[-1][-1], vertical_mat[-1][-1], diagonal_mat[-1][-1])



In [33]:
def GlobalAllignmentWithAffineGap(seq1, seq2, table, o_p, g_p):
    all_graph, max_score = GlobalAllignmentWithAffineGapGraph(seq1,seq2,table,o_p, g_p)
    print(max_score)
    ConstructGlobalAllignment(all_graph,seq1, seq2)

In [36]:
path="/Users/marcisin/Downloads/dataset_249_8 (1).txt"
s1, s2 =read_seq_file(path)
s1="AHRQPQ"
s2 = "AHED"
open_pen = 11
gap_pen = 1
table = read_BLOSUM_table()
GlobalAllignmentWithAffineGap(s1, s2, table, open_pen, gap_pen)

0
AHRQPQ
AH-ED-
